In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error


from sklearn import __version__
__version__

'1.1.3'

/Users/william/opt/anaconda3/lib/python3.9/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


{'load_extensions': {'nbextensions_configurator/config_menu/main': True,
  'contrib_nbextensions_help_item/main': True},
 'ruler_column': [80]}

sources:
https://www.youtube.com/watch?v=9yl6-HEY7_s


In [6]:
df = pd.read_csv('../datasets/train.csv')
df.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

##### Using Column Transformer

In [7]:
# drop columns
df.drop(columns = ['PID','Misc Feature','Alley','Fence','Pool QC', 'Pool Area'], axis = 1, inplace = True)

In [12]:
from AMES_Functions import pth_percentile_na

In [13]:
pth_percentile_na(df,10)

['Lot Frontage: 330',
 'Bsmt Exposure: 58',
 'Fireplace Qu: 1000',
 'Garage Type: 113',
 'Garage Yr Blt: 114',
 'Garage Finish: 114',
 'Garage Qual: 114',
 'Garage Cond: 114']

In [8]:
# get non numeric column names

df.select_dtypes(include = object).columns

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,...,Electrical,Kitchen Qual,Functional,Fireplace Qu,Garage Type,Garage Finish,Garage Qual,Garage Cond,Paved Drive,Sale Type
0,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,...,SBrkr,Gd,Typ,NaN,Attchd,RFn,TA,TA,Y,WD
1,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,...,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD
2,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,SBrkr,Gd,Typ,NaN,Detchd,Unf,TA,TA,Y,WD
3,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,...,SBrkr,TA,Typ,NaN,BuiltIn,Fin,TA,TA,Y,WD
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,...,SBrkr,TA,Typ,NaN,Detchd,Unf,TA,TA,N,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,Timber,Norm,Norm,...,SBrkr,Gd,Typ,Gd,Attchd,Fin,TA,TA,Y,WD
2047,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,...,SBrkr,TA,Typ,NaN,Detchd,Unf,TA,TA,Y,WD
2048,RL,Pave,Reg,Bnk,AllPub,Inside,Gtl,Crawfor,Norm,Norm,...,SBrkr,TA,Typ,TA,Detchd,Unf,Fa,Fa,Y,WD
2049,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,WD


In [3]:
ohe = OneHotEncoder(
    drop = 'if_binary',
    handle_unknown = 'ignore',
    sparse = False
)

In [4]:
ctx = ColumnTransformer(
    transformers = [
        #('alias_first', actual_transformer, list_of_cols_to_apply_to)
        ('one_hot', ohe, ['sex', 'smoker', 'day', 'time'])
    ], remainder = 'passthrough', #don't drop the others
    verbose_feature_names_out = False
)

In [ ]:
ctx

In [5]:
#transform train
X_train_enc = ctx.transform(X_train_filled)

#transform test
X_test_enc = ctx.transform(X_test_filled)

#recreate dataframes
X_train_enc = pd.DataFrame(X_train_enc, columns = ctx.get_feature_names_out())
X_test_enc = pd.DataFrame(X_test_enc, columns = ctx.get_feature_names_out())
X_train_enc


NameError: name 'X_train_filled' is not defined